In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U datasets
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U loralib
!pip install -q -U einops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 24.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 6.4 MB/s eta 0:00:00:00:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 2.0 MB/s eta 0:00:00


In [64]:
import torch
from transformers import (
    AutoConfig, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
)
from peft import (
    LoraConfig, PeftConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


FINETUNED_MODEL = "thangquang09/vinallama_math_solver_7B"

config = PeftConfig.from_pretrained(FINETUNED_MODEL)

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, FINETUNED_MODEL)
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = model.config.pad_token_id
generation_config.eos_token_id = model.config.eos_token_id

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [54]:
IN_CONTEXT_PROMPT = {
    "2": "Bạn là một chuyên gia về toán học. Trả lời câu hỏi sau bằng cách đưa ra đáp án chính xác nhất. Đáp án sẽ là một trong các lựa chọn A, B, C, D. Hãy suy nghĩ từng bước một."
    ,"1": "Bạn là một chuyên gia về toán học. Trả lời câu hỏi sau bằng cách đưa ra đáp án chính xác nhất. Hãy suy nghĩ từng bước một."
}

PROMTP_ANS_FORMAT = """
<|im_start|>system
{}
<|im_end|>
<|im_start|>user
{}
<|im_end|>
<|im_start|>assistant
"""

def make_ans_prompt(question, choices=None):
    if choices is not None:
        in_context_key = "2"
        choices = f"""### Các lựa chọn\n{choices}
        """
    else:
        in_context_key = "1"
        choices = ""
    
    instruction = question + "\n" + choices
    instruction = instruction.strip()
    
    prompt = PROMTP_ANS_FORMAT.format(
        IN_CONTEXT_PROMPT[in_context_key],
        instruction
    )

    return prompt

In [34]:
print(make_prompt("1 nhân 2 bằng bao nhiêu?"))


<|im_start|>system
Bạn là một chuyên gia về toán học. Trả lời câu hỏi sau bằng cách đưa ra đáp án chính xác nhất. Hãy suy nghĩ từng bước một.
<|im_end|>
<|im_start|>user
1 nhân 2 bằng bao nhiêu?
<|im_end|>
<|im_start|>assistant



In [ ]:
import re

def remove_duplicate_sentences(text):
    sentences = re.split(r'(?<=[.!?])\s+', text)  # Tách câu dựa trên dấu câu
    seen = set()
    filtered_sentences = []
    
    for sentence in sentences:
        if sentence not in seen:  # Chỉ thêm câu nếu nó chưa xuất hiện trước đó
            filtered_sentences.append(sentence)
            seen.add(sentence)
    
    return " ".join(filtered_sentences)

In [55]:
def inference(tokenizer, model, question, choices, generation_config, device="cpu"):
    prompt = make_ans_prompt(question, choices)
    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config
        )
    ans = tokenizer.decode(outputs[0], skip_special_tokens=True)
    processed_ans = remove_duplicate_sentences(ans)
    processed_ans = processed_ans.split("<|im_start|> assistant")
    return processed_ans[1]

In [72]:
question = """
Bella đã mua tem tại bưu điện. Một số tem có thiết kế tuyết rơi, một số có thiết kế xe tải và một số có thiết kế hoa hồng. Bella đã mua 11 tem tuyết rơi. Cô ấy đã mua 9 tem xe tải nhiều hơn số tem tuyết rơi và ít hơn 13 tem hoa hồng so với số tem xe tải. Bella đã mua tổng cộng bao nhiêu tem?
""".strip()

choices = """"""

choices = choices if choices != """""" else None

print(question)
print(choices)

Bella đã mua tem tại bưu điện. Một số tem có thiết kế tuyết rơi, một số có thiết kế xe tải và một số có thiết kế hoa hồng. Bella đã mua 11 tem tuyết rơi. Cô ấy đã mua 9 tem xe tải nhiều hơn số tem tuyết rơi và ít hơn 13 tem hoa hồng so với số tem xe tải. Bella đã mua tổng cộng bao nhiêu tem?
None


In [73]:
device = "cuda" if torch.cuda.is_available() else "cpu"

ans = inference(tokenizer, model, question, choices, generation_config, device="cuda")
print(ans)


Bella đã mua 20 tem xe tải (11 + 9). Cô ấy đã mua 7 tem hoa hồng (20 - 13). Bella đã mua tổng cộng 38 tem (11 + 20 + 7). Bella đã mua tổng cộng 38 tem. Câu trả lời là: 38 tem. Đáp án là 38

Chọn đáp án đúng
Bella đã mua tổng cộng 38 tem. Đáp án là 38

Chọn đáp án đúng
Bella đã mua
